<a href="https://colab.research.google.com/github/Dalal7/Covid19-Global-Dashboard/blob/main/COVID_19_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**COVID-19 Data Processing**</br>
In this file, I will process and clean datasets for COVID-19 cases across the world. The objective is to get a clean dataset that can be used as the source of a dashboard. The datasets comes from COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University. [Data Source](https://github.com/CSSEGISandData/COVID-19/blob/master/) 

In [1]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
from math import sqrt
from datetime import datetime

In [2]:
path = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/'
raw_data_confirmed = pd.read_csv(path+'time_series_covid19_confirmed_global.csv?raw=true')
raw_data_deaths = pd.read_csv(path+'time_series_covid19_deaths_global.csv?raw=true')
raw_data_recovered = pd.read_csv(path+ 'time_series_covid19_recovered_global.csv?raw=true')

In [3]:
raw_data_confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,46837,47072,47306,47516,47716,47851,48053,48116,48229,48527,48718,48952,49161,49378,49621,49681,49817,50013,50190,50433,50655,50810,50886,51039,51280,51350,51405,51526,51526,51526,51526,53011,53105,53105,53207,53332,53400,53489,53538,53584
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,41302,42148,42988,43683,44436,45188,46061,46863,47742,48530,49191,50000,50637,51424,52004,52542,53003,53425,53814,54317,54827,55380,55755,56254,56572,57146,57727,58316,58316,58991,59438,59623,60283,61008,61705,62378,63033,63595,63971,64627
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,86730,87502,88252,88825,89416,90014,90579,91121,91638,92102,92597,93065,93507,93933,94371,94781,95203,95659,96069,96549,97007,97441,97857,98249,98631,98988,99311,99610,99897,100159,100408,100645,100873,101120,101382,101657,101913,102144,102369,102641
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6955,7005,7050,7084,7127,7162,7190,7236,7288,7338,7382,7382,7446,7466,7519,7560,7577,7602,7633,7669,7699,7756,7806,7821,7875,7919,7983,8049,8117,8166,8192,8249,8308,8348,8348,8489,8586,8586,8586,8682
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,15493,15536,15591,15648,15729,15804,15925,16061,16161,16188,16277,16362,16407,16484,16562,16626,16644,16686,16802,16931,17029,17099,17149,17240,17296,17371,17433,17553,17568,17608,17642,17684,17756,17864,17974,18066,18156,18193,18254,18343


In [4]:
print("The Shape of Cornirmed is: ", raw_data_confirmed.shape)
print("The Shape of Deaths is: ", raw_data_deaths.shape)
print("The Shape of Recovered is: ", raw_data_recovered.shape)

The Shape of Cornirmed is:  (272, 361)
The Shape of Deaths is:  (272, 361)
The Shape of Recovered is:  (257, 361)


In [5]:
# Checking and counting missing data
raw_data_confirmed.isnull().sum()

Province/State    188
Country/Region      0
Lat                 1
Long                1
1/22/20             0
                 ... 
1/8/21              0
1/9/21              0
1/10/21             0
1/11/21             0
1/12/21             0
Length: 361, dtype: int64

In [6]:
print(raw_data_confirmed[raw_data_confirmed['Long'].isnull()])

            Province/State Country/Region  Lat  ...  1/10/21  1/11/21  1/12/21
52  Repatriated Travellers         Canada  NaN  ...       13       13       13

[1 rows x 361 columns]


In [7]:
# Filling the missing values of Canada Long and Lat 
raw_data_confirmed = raw_data_confirmed.fillna({'Long':106.3468 ,'Lat':56.1304})
raw_data_confirmed.isnull().sum()

Province/State    188
Country/Region      0
Lat                 0
Long                0
1/22/20             0
                 ... 
1/8/21              0
1/9/21              0
1/10/21             0
1/11/21             0
1/12/21             0
Length: 361, dtype: int64

In [10]:
# Filling missing Province/State with Country/Region
raw_data_confirmed['Province/State'].fillna(raw_data_confirmed['Country/Region'], inplace=True)
raw_data_deaths['Province/State'].fillna(raw_data_deaths['Country/Region'], inplace=True)
raw_data_recovered['Province/State'].fillna(raw_data_recovered['Country/Region'], inplace=True)

raw_data_confirmed.isnull().sum()

Province/State    0
Country/Region    0
Lat               0
Long              0
1/22/20           0
                 ..
1/8/21            0
1/9/21            0
1/10/21           0
1/11/21           0
1/12/21           0
Length: 361, dtype: int64

In [11]:
# unpivots the dataframes from wide format to long format
raw_data_confirmed2 = pd.melt(raw_data_confirmed, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
raw_data_deaths2 = pd.melt(raw_data_deaths, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
raw_data_recovered2 = pd.melt(raw_data_recovered, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])

In [12]:
print("The Shape of Cornirmed is: ", raw_data_confirmed2.shape)
print("The Shape of Deaths is: ", raw_data_deaths2.shape)
print("The Shape of Recovered is: ", raw_data_recovered2.shape)

The Shape of Cornirmed is:  (97104, 6)
The Shape of Deaths is:  (97104, 6)
The Shape of Recovered is:  (91749, 6)


In [14]:
# Renaming the new columns
raw_data_confirmed2.columns = raw_data_confirmed2.columns.str.replace('value', 'Confirmed')
raw_data_deaths2.columns = raw_data_deaths2.columns.str.replace('value', 'Deaths')
raw_data_recovered2.columns = raw_data_recovered2.columns.str.replace('value', 'Recovered')

In [15]:
raw_data_recovered2.head()

,Province/State,Country/Region,Lat,Long,Date,Recovered
0,Afghanistan,Afghanistan,33.93911,67.709953,1/22/20,0
1,Albania,Albania,41.15330,20.168300,1/22/20,0
2,Algeria,Algeria,28.03390,1.659600,1/22/20,0
3,Andorra,Andorra,42.50630,1.521800,1/22/20,0
4,Angola,Angola,-11.20270,17.873900,1/22/20,0


In [16]:
# Joining the data

# Confirmed with Deaths
full_join = raw_data_confirmed2.merge(raw_data_deaths2[['Province/State','Country/Region','Date','Deaths']], 
                                      how = 'left', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

print("Shape of first join: ", full_join.shape)

# full join with Recovered
full_join = full_join.merge(raw_data_recovered2[['Province/State','Country/Region','Date','Recovered']], 
                                      how = 'left', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

print("Shape of second join: ", full_join.shape)

full_join.head()

Shape of first join:  (97104, 7)
Shape of second join:  (97104, 8)


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Afghanistan,Afghanistan,33.93911,67.709953,1/22/20,0,0,0.0
1,Albania,Albania,41.15330,20.168300,1/22/20,0,0,0.0
2,Algeria,Algeria,28.03390,1.659600,1/22/20,0,0,0.0
3,Andorra,Andorra,42.50630,1.521800,1/22/20,0,0,0.0
4,Angola,Angola,-11.20270,17.873900,1/22/20,0,0,0.0


In [17]:
full_join.isnull().sum()

Province/State       0
Country/Region       0
Lat                  0
Long                 0
Date                 0
Confirmed            0
Deaths               0
Recovered         5712
dtype: int64

In [18]:
# Adding Month and Year as a new Column
full_join['Date'] = pd.to_datetime(full_join['Date'])
full_join['Month-Year'] = full_join['Date'].dt.strftime('%b-%Y')

In [19]:
full_join.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year
0,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0.0,Jan-2020
1,Albania,Albania,41.15330,20.168300,2020-01-22,0,0,0.0,Jan-2020
2,Algeria,Algeria,28.03390,1.659600,2020-01-22,0,0,0.0,Jan-2020
3,Andorra,Andorra,42.50630,1.521800,2020-01-22,0,0,0.0,Jan-2020
4,Angola,Angola,-11.20270,17.873900,2020-01-22,0,0,0.0,Jan-2020


In [20]:
# Since the number of cases is cumulative, we have to subtract the number of cases of the previous date

#creating a new df    
full_join2 = full_join.copy()

#creating a new date columns - 1
full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1',
                          'Date': 'Date Minus 1'}, inplace=True)

#Joing on the 2 DFs
full_join3 = full_join.merge(full_join2[['Province/State', 'Country/Region','Confirmed - 1', 'Deaths - 1', 
                            'Recovered - 1', 'Date - 1', 'Date Minus 1']], how = 'left',
                             left_on = ['Province/State','Country/Region','Date'], 
                             right_on = ['Province/State', 'Country/Region','Date - 1'])
# Additional Calculations
full_join3['Confirmed Daily'] = full_join3['Confirmed'] - full_join3['Confirmed - 1']
full_join3['Deaths Daily'] = full_join3['Deaths'] - full_join3['Deaths - 1']
full_join3['Recovered Daily'] = full_join3['Recovered'] - full_join3['Recovered - 1']

full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed - 1,Deaths - 1,Recovered - 1,Date - 1,Date Minus 1,Confirmed Daily,Deaths Daily,Recovered Daily
0,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
1,Albania,Albania,41.15330,20.168300,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
2,Algeria,Algeria,28.03390,1.659600,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
3,Andorra,Andorra,42.50630,1.521800,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
4,Angola,Angola,-11.20270,17.873900,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN


In [21]:
print(full_join3.shape)

(97104, 17)


In [22]:
# Additing manually the numbers for first day

full_join3['Confirmed Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Confirmed']
full_join3['Deaths Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Deaths']
full_join3['Recovered Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Recovered']

# deleting columns
del full_join3['Confirmed - 1']
del full_join3['Deaths - 1']
del full_join3['Recovered - 1']
del full_join3['Date - 1']
del full_join3['Date Minus 1']

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [23]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed Daily,Deaths Daily,Recovered Daily
0,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0.0,Jan-2020,0.0,0.0,0.0
1,Albania,Albania,41.15330,20.168300,2020-01-22,0,0,0.0,Jan-2020,0.0,0.0,0.0
2,Algeria,Algeria,28.03390,1.659600,2020-01-22,0,0,0.0,Jan-2020,0.0,0.0,0.0
3,Andorra,Andorra,42.50630,1.521800,2020-01-22,0,0,0.0,Jan-2020,0.0,0.0,0.0
4,Angola,Angola,-11.20270,17.873900,2020-01-22,0,0,0.0,Jan-2020,0.0,0.0,0.0


In [24]:
full_join3.to_csv('CoronaVirus_PowerBI_Raw.csv')